In [130]:
import os

import h5py
import matplotlib.pyplot as plt
import numpy as np
import oculus.research.orcoptics.storage_async as storage_async
import torch
import torch.fft as fft
from matplotlib import pyplot as plt
from scipy.fft import fft2, fftshift, ifft2, ifftshift
from scipy.stats import norm


def circ(x, y):
    return np.where(np.sqrt(x**2 + y**2) <= 0.5, 1, 0)


###key code: fft-propagation-fft, return electric field
def wvprop(amp, phi, distance, delta, wv):
    us = amp * np.exp(1j * phi)
    # sy, sx = amp.shape
    # # # original equation
    # delta=0.5
    # as_ = fftshift(fft2(ifftshift(us))) #has one more shift
    # # print("shape as_", as_.shape)
    # fx = np.linspace(-1, 1, sx) / (2 * delta)  # has L elements； Julian fx and fy
    # fy = np.linspace(-1, 1, sy) / (2 * delta)
    # fx, fy = np.meshgrid(fx, fy)
    # r2 = fx**2 + fy**2
    # # asprop = as_ * np.exp(1j * 2 * np.pi / wv * distance * np.sqrt(1 - wv**2 * r2))
    # asprop = as_ * np.exp(1j * 2 * np.pi / wv * distance * (1 -0.5* wv**2 * r2))
    # us2 = ifftshift(ifft2(fftshift(asprop)))

    # # #equation and the mesh from book, alternative check, WIP
    sy, sx = amp_pad.shape  #!!!!!!!! should be amp_padding shape
    sampling_criteria = wv * distance / (delta * size_img)
    k = 2 * np.pi / wv
    # if distance == 0:
    #     us2 = us
    # else:
    #     if sampling_criteria >= 1:
    #         method = "IR"  # fresnel inpulse response
    #         print("IR")
    #     else:
    #         method = "TF"  # Fresnel transfer function
    #         print("TF")
    #     if method == "IR":
    #         # x1 = -L1/2:dx1:L1/2-dx1
    #         delta=size_img/sx
    #         x= np.arange(- size_img/ 2 , size_img /2 , delta)  # range span: max is 1/dx
    #         x, y = np.meshgrid(x, x)
    #         h = (
    #             1
    #             / (1j * wv * distance)
    #             * np.exp((1j * k / (2 * distance) )* (x**2 + y**2))
    #         )  # Minchul old

    #         U2 = fft2(fftshift(h)) * delta**2 * fft2(fftshift(us))
    #         us2 = ifftshift(ifft2(U2))
    #     elif method == "TF":
    #         # fx=-1/(2*dx1):1/L1:1/(2*dx1)-1/L1
    #delta=size_img/sx
    delta=0.5
    as_ = fftshift(fft2(ifftshift(us))) #has one more shift
    # fx = np.arange(-1 / (2 * delta), 1 / (2 * delta), 1 / size_img)  # range span: max is 1/dx; should consider cover the CRA
    # fx, fy = np.meshgrid(fx, fx)
    fx = np.linspace(-1, 1, sx) / (2 * delta)  # has L elements； Julian fx and fy
    fy = np.linspace(-1, 1, sy) / (2 * delta)
    fx, fy = np.meshgrid(fx, fy)
    H = np.exp(-1j * np.pi * wv * distance * (fx**2 + fy**2))  # Minchul old
    U2 =  as_*H
    us2 = ifftshift(ifft2(fftshift(U2)))
    return us2


def binning(A, bs):
    s = A.shape
    B = np.reshape(A.flatten(), bs[0], s[0] // bs[0], [])
    B = np.reshape(np.transpose(B, [0, 2, 1]), bs[0], bs[1], [])
    B = np.sum(B, axis=(0, 1))
    B = np.reshape(B, s[:2])
    return B


def getNsr(img):
    nsr = np.std(img, keepdims=True) / np.mean(img, keepdims=True)
    return nsr


def generate_2d_gaussian(sigma, center2, size2):
    # Define the two-dimensional grid
    sizex, sizey = size2
    centerx, centery = center2
    x, y = np.meshgrid(
        np.linspace(-(sizey - 1) / 2 - centery, (sizey - 1) / 2 - centery, sizey),
        np.linspace(-(sizex - 1) / 2 - centerx, (sizex - 1) / 2 - centerx, sizex),
    )
    # Calculate the distances from the center of the grid
    d = np.sqrt(x * x + y * y)
    # Define the standard deviation and calculate the 2D Gaussian matrix
    g = np.exp(-((d) ** 2 / (2 * sigma**2)))
    g = g / np.sum(g)
    return g


def gaussmf(x, params):
    sigma, mean = params
    return norm.pdf(x, loc=mean, scale=sigma)

In [126]:
sy, sx = amp.shape
wv=0.515
distance=2000
# if distance == 0:
#     us2 = us
# else:
#     if sampling_criteria >= 1:
#         method = "IR"  # fresnel inpulse response
#         print("IR")
#     else:
#         method = "TF"  # Fresnel transfer function
#         print("TF")
#     if method == "IR":
#         # x1 = -L1/2:dx1:L1/2-dx1
#         delta=size_img/sx
#         x= np.arange(- size_img/ 2 , size_img /2 , delta)  # range span: max is 1/dx
#         x, y = np.meshgrid(x, x)
#         h = (
#             1
#             / (1j * wv * distance)
#             * np.exp((1j * k / (2 * distance) )* (x**2 + y**2))
#         )  # Minchul old

#         U2 = fft2(fftshift(h)) * delta**2 * fft2(fftshift(us))
#         us2 = ifftshift(ifft2(U2))
#     elif method == "TF":
#         # fx=-1/(2*dx1):1/L1:1/(2*dx1)-1/L1
#delta=size_img/sx
delta=0.5
fx1 = np.arange(-1 / (2 * delta), 1 / (2 * delta), 1 / size_img)  # range span: max is 1/dx; should consider cover the CRA
print(fx1[500])
print(len(fx1))
fx2 = np.linspace(-1, 1, sx) / (2 * delta)  # has L elements； Julian fx and fy
print(fx2[500])
print(len(fx2))


In [36]:
wv=0.515
#test
wv_list = wv_sim_num_group['r']
wv_list = np.linspace(0.638 -  0.002, 0.638 +  0.002, 80)
len(wv_list)


In [81]:
import numpy as np
# Main code
wv_sim_group = {"r": 0.638, "g": 0.520, "b": 0.450}
wv_sim_group = {"g": 0.520} #only use green
# wv_sim_num_group = {"r": 80, "g": 80, "b": 80}
# linewidth_group = {"r": 0.002, "g": 0.001, "b": 0.002}
wv_sim_num_group = {"r": 80, "g": 80, "b": 80}
linewidth_group = {"r": 0.002, "g": 0.001, "b": 0.002}
# pitch x, x is the ridge direction.
pitch_x = 20
pitch_y = pitch_x/2
emitter_size = 0.707  # emitter_size: sigma of the Gaussian-shape emitter in um.
n = 1.5  # index of the pic WG.
delta = 0.5  # simulation resolution in um
size_img=300 #um
rep_num_x = int(size_img // pitch_x)  #the number of emitters along x
rep_num_y = int(size_img // pitch_y)  #the number of emitters along y
peroid = 10  # number of ridges driven at different frequencies.
#will check the GDS later
routing_right_phase=4000*n
routing_right_phase=3800*n

# padding = 352+20  # padding; this means adding padding/2 surrounding at 4 directions
padding=900
padding=1000

Ly = int(pitch_y / delta)  #Lx, Ly are the matrix size of the simulation area for one emitter.
Lx = int(pitch_x / delta)
amp = np.zeros(
    (Ly * rep_num_y, Lx * rep_num_x)
)  #amp is the amplitude map at the emitter plane. total number of grids

#### Randomness##############
randomness=0.125 #(+-) in fact its 50%
randx = randomness*pitch_x  #um? #the amplitude of the random displacement along x.
rand_x_list = np.random.rand(rep_num_y, rep_num_x) * randx*2  - randx  # Random x-displacement for each element; hyperuniform
rand_y_list = (np.random.rand(rep_num_y, 1) * randx*2 - randx).repeat(rep_num_x, axis=1)  # Random y-displacement per row
# rand_x_list = np.cumsum(np.random.rand(rep_num_y, rep_num_x) * randx * 2 - randx, axis=1)  # Cumulative random x-displacement for each element
# rand_y_list = np.cumsum(np.random.rand(rep_num_y, 1) * randx * 2 - randx, axis=0).repeat(rep_num_x, axis=1)  # Cumulative random y-displacement per row
for i in range(rep_num_y):
    for j in range(rep_num_x):
        amp[Ly * i : Ly * (i + 1), Lx * j : Lx * (j + 1)] = generate_2d_gaussian(
            emitter_size / delta, [rand_y_list[i, j] / delta, rand_x_list[i, j] / delta], [Ly, Lx]
        )
amp_pad = np.pad(amp, pad_width=padding)
# Set the figure size

# assigning phase distribution;  phi_add_x_base:
# X, Y = np.meshgrid(
#     np.arange(1, rep_num_x + 1) * pitch_x, np.arange(1, rep_num_y + 1) * pitch_y
# )
# opd_list = (X + Y) * n
# opd_list += n * rand_x_list
# plt.imshow(opd_list, cmap="gray")
# X, Y = np.meshgrid(
#     np.arange(1, rep_num_x + 1) * pitch_x, np.arange(1, rep_num_y + 1) * pitch_y
# )
# Create a 2D array with alternating rows of increasing and decreasing phase

phase_increases = np.arange(rep_num_x) * (n*pitch_x)
phase_decreases = (rep_num_x-np.arange(rep_num_x)-1) * (n*pitch_x)
opd_list = np.zeros((rep_num_y, rep_num_x))
for i in range(rep_num_y):
    if i % 2 == 0:  
        opd_list[i] = phase_increases+(i//2*(2*pitch_x+6))*n
        opd_list[i] = phase_increases+(i//2*(26.4))*n
    else:  
        opd_list[i] = phase_decreases+(i//2*(2*pitch_x+6))*n+routing_right_phase
        opd_list[i] = phase_decreases+(i//2*(26.4))*n+routing_right_phase

print(opd_list) #in length of air
# assigning additonal phase from active phase modulation. phi_add_x_list stores the list of phase multiplier that is going to use in the simulation.
# phi_add_x_base is the base of the active phase for each frame. Here, assuming different rows are oscilating at 0, 1f, 2f, 3f, ... towards (peroid-1)f and repeat.
# phi_add_x_list = np.linspace(2*np.pi/peroid,2*np.pi,peroid)
# phi_add_x_base = np.kron(np.ones(rep_num_y // peroid), np.arange(peroid))
# phi_add_x_base = np.kron(np.ones(rep_num_x),np.transpose([phi_add_x_base]))
# phi_add_x_list = [0]
# phi_add_x_base = [0]
#print(phi_add_x_base)
#print(phase_increases)
#print(phase_decreases)


figsize = (5, 5)
plt.figure(figsize=figsize)
plt.imshow(amp, cmap="gray")
plt.axis("off")
plt.show()
figsize = (5, 5)
plt.figure(figsize=figsize)
plt.imshow(opd_list, cmap="gray")
figsize = (5, 5)
plt.figure(figsize=figsize)
plt.imshow(amp_pad, cmap="gray")
plt.axis("off")
plt.show()

In [42]:
wv = wv_sim_group['g']
#print(wv)
wv_sim_num = wv_sim_num_group['g']
linewidth = linewidth_group['g']

# initializing the field distribution.
us_list = np.zeros((len(phi_add_x_list),wv_sim_num,Ly * rep_num_y + 2 * padding, Lx * rep_num_x + 2 * padding),dtype=np.complex_)
# initializing the intensity distribution at LCoS
# getting the wavelength list and weight
wv_list = np.linspace(wv -  linewidth, wv + linewidth, wv_sim_num)
wv_coef = gaussmf(wv_list, [linewidth / 2, wv])
wv_coef = wv_coef / np.sum(wv_coef)
plt.figure(figsize=(5, 5))
plt.plot(wv_list, wv_coef, label='Gaussian Weight', color='blue')
plt.title('Wavelength vs. Gaussian Coefficient')
plt.xlabel('Wavelength (nm)'); plt.ylabel('Coefficient')
plt.show()

In [2]:
#test area
a=3
def sumtest(a, b):
    return a+b+c
c=5
sumtest(1,1)

lambda_ =0.5
phi = 2 * np.pi / lambda_ * opd_list
# print(phi)
print( )
# getting the phase map
phi = np.kron(phi, np.ones((Ly, Lx)))
print(phi)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Load data from a text file
data = np.loadtxt('/Users/qingjunwang/Downloads/green.txt', delimiter=',')  # Adjust the delimiter based on your file

# Extract x and y coordinates
x = data[:, 0]
y = data[:, 1]

# Create a plot
plt.figure(figsize=(8, 6))
plt.plot(x, y, marker='o', linestyle='-', color='b')  # Customize the plot with markers, line styles, and colors
plt.title('Curve from Text File')
plt.xlabel('X Axis Label')
plt.ylabel('Y Axis Label')
plt.grid(True)
plt.show()

## sum_intensity_calculator

In [10]:
def sum_intensity_calculator(distance_array,wv_sim_group,wv_sim_num_group,linewidth_group,intensity_arr,use_array):
    import time
    start_time = time.time()

    sum_distance = np.zeros((Ly * rep_num_y + 2 * padding, Lx * rep_num_x + 2 * padding))
    imcolor_sum = np.zeros((Ly * rep_num_y + 2 * padding, Lx * rep_num_x + 2 * padding))
    k=0
    for distance in distance_array:
        for cc, c in enumerate(wv_sim_group): # select r/g/b
            wv = wv_sim_group[c]
            #print(wv)
            wv_sim_num = wv_sim_num_group[c]
            linewidth = linewidth_group[c]

            # initializing the field distribution.
            us_list = np.zeros((len(phi_add_x_list),wv_sim_num,Ly * rep_num_y + 2 * padding, Lx * rep_num_x + 2 * padding),dtype=np.complex_)
            # initializing the intensity distribution at LCoS
            # getting the wavelength list and weight
            wv_list = np.linspace(wv -  linewidth, wv + linewidth, wv_sim_num)
            wv_coef = gaussmf(wv_list, [linewidth / 2, wv])
            wv_coef = wv_coef / np.sum(wv_coef)
            # # Load wavelength data from a text file
            # data = np.loadtxt('green.txt', delimiter=',')  # Adjust the delimiter based on your file
            # wv_list = data[:, 0]
            # wv_coef = data[:, 1]/np.sum(data[:, 1])
            # print("load laser data from txt")

            # plt.figure(figsize=(10, 5))
            # plt.plot(wv_list, wv_coef, label='Gaussian Weight', color='blue')
            # plt.title('Wavelength vs. Gaussian Coefficient')
            # plt.xlabel('Wavelength (nm)'); plt.ylabel('Coefficient')
            # plt.show()
                                # sum for situation at different addtional phases.
                                # for j in range(len(phi_add_x_list)):
            imwv_list = np.zeros((len(wv_list), Ly * rep_num_y + 2 * padding, Lx * rep_num_x + 2 * padding))
                                #phi_add_x = phi_add_x_list[j] * phi_add_x_base

            # sum for situation at different wavelengths
            for i in range(len(wv_list)):
                lambda_ = wv_list[i]
                #phi = 2 * np.pi / lambda_ * opd_list + phi_add_x
                phi = 2 * np.pi / lambda_ * opd_list 
                # getting the phase map
                phi = np.kron(phi, np.ones((Ly, Lx)))
                # padding
                phi_pad = np.pad(phi, pad_width=padding)
                # angular spectrum algorithm
                us = wvprop(amp_pad, phi_pad, distance, delta, lambda_)
                # storing the results
                imwv_list[i, :, :] = np.abs(us) ** 2 #intensity I of electric field
                # us_list[j ,i ,:, :] = us * np.sqrt(wv_coef[i])
            imwv_sum = np.sum(wv_coef.reshape(len(wv_coef), 1, 1) * imwv_list, axis=0) #sum of single color Intensity
            imcolor_sum+=imwv_sum #sum of multiple color, this layer is finished
            # imphi_sum += imwv_sum
        if use_array==1:
            sum_distance+=imcolor_sum*intensity_arr[k] #sum of multiple plane
            print("intensity_arr[k]=", intensity_arr[k])
        if use_array==0:
            sum_distance+=imcolor_sum #sum of multiple plane
        k+=1
        print("k=",k)
    end_time = time.time()
    running_time = end_time - start_time
    print(f"Running time: {running_time} seconds")
    return sum_distance
    # us_list = us_list.reshape((len(phi_add_x_list)*wv_sim_num,Ly * rep_num_y + 2 * padding, Lx * rep_num_x + 2 * padding))
    # size = len(us_list[0])
    # # cutting the edge
    # #us_list = us_list[:,size//3:size//3*2,size//3:size//3*2]
    # us_list = us_list[:,:]
    # us_group[c] = us_list

## sum_electric_field_calculator


In [11]:
def sum_electric_field_calculator(
    distance_array,
    wv_sim_group,
    wv_sim_num_group,
    linewidth_group,
    intensity_arr,
    use_array):
    import time

    start_time = time.time()

    imcolor_sum = np.zeros((Ly * rep_num_y + 2 * padding, Lx * rep_num_x + 2 * padding))
    for cc, c in enumerate(wv_sim_group):  # select r/g/b
        wv = wv_sim_group[c]
        wv_sim_num = wv_sim_num_group[c]
        linewidth = linewidth_group[c]
        # getting the wavelength list and weight for each color
        wv_list = np.linspace(wv - linewidth, wv + linewidth, wv_sim_num)
        wv_coef = gaussmf(wv_list, [linewidth / 2, wv])
        wv_coef = wv_coef / np.sum(wv_coef)
        # # Load wavelength data from a text file
        # data = np.loadtxt('green.txt', delimiter=',')  # Adjust the delimiter based on your file
        # wv_list = data[:, 0]
        # wv_coef = data[:, 1]/np.sum(data[:, 1])
        # print("load laser data from txt")

        # initializing the intensity distribution at LCoS
        imwv_list = np.zeros(
            (len(wv_list), Ly * rep_num_y + 2 * padding, Lx * rep_num_x + 2 * padding)
        )

        # sum for situation at different subpeaks
        for i in range(len(wv_list)):
            lambda_ = wv_list[i]
            # phi = 2 * np.pi / lambda_ * opd_list + phi_add_x
            phi = 2 * np.pi / lambda_ * opd_list
            # getting the phase map
            phi = np.kron(phi, np.ones((Ly, Lx)))
            #print(phi)
            # padding
            phi_pad = np.pad(phi, pad_width=padding)
            # create array to store the sum of electric field at different z planes
            # Create an array of zeros with the same shape as us2
            us2 = wvprop(amp_pad, phi_pad, 80, delta, lambda_)
            us_sum = np.zeros_like(us2)
            k = 0
            for distance in distance_array:
                us = wvprop(
                    amp_pad, phi_pad, distance, delta, lambda_
                )  # angular spectrum algorithm
                if use_array == 0:
                    us_sum += us
                if use_array == 1:
                    us_sum += us * np.sqrt(intensity_arr[k])
                    # print("distantce",distance, "intensity_arr[k]=", intensity_arr[k])
                k += 1
                # print("k=",k)
            # storing the results
            imwv_list[i, :, :] = (
                np.abs(us_sum) ** 2
            )  # intensity I of a single wavelength
            # us_list[j ,i ,:, :] = us * np.sqrt(wv_coef[i])
        imwv_sum = np.sum(
            wv_coef.reshape(len(wv_coef), 1, 1) * imwv_list, axis=0
        )  # sum of single color Intensity
        imcolor_sum += imwv_sum  # sum of multiple color, this layer is finished
        # imphi_sum += imwv_sum
    end_time = time.time()
    running_time = end_time - start_time
    print(f"Running time: {running_time} seconds")
    return imcolor_sum
    # us_list = us_list.reshape((len(phi_add_x_list)*wv_sim_num,Ly * rep_num_y + 2 * padding, Lx * rep_num_x + 2 * padding))
    # size = len(us_list[0])
    # # cutting the edge
    # #us_list = us_list[:,size//3:size//3*2,size//3:size//3*2]
    # us_list = us_list[:,:]
    # us_group[c] = us_list


    #remember using this in imgae plot
    plt.imshow(implot_scaled, cmap="gray", vmin=0, vmax=255) ###highlight!


## plot_contrast_and_save

In [96]:
def plot_contrast_and_save(implot, plane_n, plane_distance, distance0,use_array,electric_field,randomness,pitch_x,note):
    import matplotlib.pyplot as plt
    import numpy as np
    from PIL import Image

    # Scale the image to max=255
    implot_scaled = (implot / np.max(implot)) * 255
    implot_scaled = implot_scaled.astype(np.uint8)
    # size_img=300
    # Calculate the starting indices to slice the middle part
    start_row = (implot.shape[0] - size_img) // 2
    start_col = (implot.shape[1] - size_img) // 2
    # Extract the middle size_img x size_img area
    implot_nonpadding = implot_scaled[start_row:start_row + size_img, start_col:start_col + size_img]
    print(f"contrast={round(float(getNsr(implot_nonpadding)),5)}")

    fig, axs = plt.subplots(1, 2, figsize=(10, 5))  # 1 row, 2 columns
    axs[0].imshow(implot_scaled, cmap='gray', vmin=0, vmax=255)
    axs[0].set_title('Full Image')
    axs[1].imshow(implot_nonpadding, cmap='gray', vmin=0, vmax=255)
    axs[1].set_title('non-padding Area')
    fig.colorbar(axs[0].imshow(implot_scaled, cmap='gray', vmin=0, vmax=255), ax=axs[0])
    fig.colorbar(axs[1].imshow(implot_nonpadding, cmap='gray', vmin=0, vmax=255), ax=axs[1])
    plt.show()

    image_pil = Image.fromarray(implot_nonpadding)
    save_path = (
        "sum_electric_field="
        +str(electric_field)
        +"_note_"
        +note
        +"-"
        +str(randomness*2)
        +"random_"
        +str(pitch_x)
        +"um_pitch_"
        +str(plane_n)
        + "plane_"
        + str(plane_distance)
        + "umdistance_"
        + str(distance0)
        + "umdistance0_"
        + str(use_array)
        + "intensityarray_"
        "contrast"
        + str(round(float(getNsr(implot_nonpadding)), 5))
        + ".png"
    )
    # save_path = str(plane_n)+'plane_'+str(plane_distance)+'umdistance_'+str(distance0)+'umdistance0_contrast'+str(float(getNsr(implot)))+'.png'
    image_pil.save(save_path)
    print(f"Image saved successfully at {save_path}")

## define intensity array of z planes

In [14]:
intensity_arr66=[0.09999999999999998, 0.08999999999999998, 0.08099999999999999, 0.07289999999999999, 0.06560999999999999, 0.059049, 0.05314409999999999, 0.047829689999999994, 0.043046721, 0.0387420489, 0.03486784401, 0.031381059609, 0.028242953648100005, 0.025418658283290003, 0.022876792454961003, 0.020589113209464903, 0.01853020188851841, 0.016677181699666574, 0.015009463529699915, 0.013508517176729925, 0.012157665459056933, 0.010941898913151239, 0.009847709021836116, 0.008862938119652504, 0.007976644307687254, 0.0071789798769185285, 0.006461081889226676, 0.0058149737003040085, 0.005233476330273608, 0.004710128697246248, 0.004239115827521623, 0.0038152042447694608, 0.0034336838202925143, 0.003090315438263263, 0.0027812838944369367, 0.0025031555049932434, 0.002252839954493919, 0.002027555959044527, 0.0018248003631400746, 0.0016423203268260671, 0.0014780882941434605, 0.0013302794647291143, 0.001197251518256203, 0.0010775263664305826, 0.0009697737297875244, 0.0008727963568087721, 0.0007855167211278949, 0.0007069650490151054, 0.0006362685441135948, 0.0005726416897022354, 0.0005153775207320119, 0.0004638397686588107, 0.0004174557917929296, 0.0003757102126136367, 0.000338139191352273, 0.0003043252722170457, 0.00027389274499534116, 0.00024650347049580705, 0.00022185312344622635, 0.0001996678111016037, 0.00017970102999144335, 0.00016173092699229901, 0.00014555783429306913, 0.0001310020508637622, 0.000117901845777386, 0.00010611166119964739]

In [40]:
intensity_arr459R99=[0.010000000000000009, 0.00990000000000001, 0.009801000000000008, 0.009702990000000009, 0.009605960100000008, 0.009509900499000008, 0.009414801494010009, 0.009320653479069907, 0.009227446944279209, 0.009135172474836417, 0.009043820750088052, 0.008953382542587172, 0.0088638487171613, 0.008775210229989686, 0.00868745812768979, 0.008600583546412893, 0.008514577710948762, 0.008429431933839274, 0.008345137614500881, 0.008261686238355874, 0.008179069375972315, 0.008097278682212591, 0.008016305895390466, 0.00793614283643656, 0.007856781408072194, 0.007778213593991473, 0.007700431458051557, 0.007623427143471042, 0.007547192872036332, 0.007471720943315968, 0.007397003733882809, 0.007323033696543981, 0.0072498033595785405, 0.007177305325982756, 0.007105532272722927, 0.007034476949995698, 0.00696413218049574, 0.006894490858690783, 0.0068255459501038755, 0.006757290490602837, 0.006689717585696808, 0.006622820409839841, 0.006556592205741442, 0.006491026283684027, 0.006426116020847187, 0.006361854860638715, 0.006298236312032328, 0.006235253948912005, 0.006172901409422885, 0.006111172395328656, 0.006050060671375369, 0.005989560064661615, 0.0059296644640149995, 0.00587036781937485, 0.005811664141181101, 0.00575354749976929, 0.0056960120247715965, 0.00563905190452388, 0.005582661385478641, 0.0055268347716238555, 0.005471566423907617, 0.005416850759668541, 0.005362682252071854, 0.005309055429551136, 0.005255964875255624, 0.005203405226503068, 0.005151371174238037, 0.005099857462495657, 0.005048858887870701, 0.004998370298991994, 0.004948386596002074, 0.004898902730042053, 0.004849913702741633, 0.004801414565714216, 0.004753400420057074, 0.004705866415856503, 0.0046588077516979375, 0.004612219674180958, 0.004566097477439149, 0.004520436502664758, 0.00447523213763811, 0.0044304798162617285, 0.004386175018099112, 0.00434231326791812, 0.0042988901352389395, 0.004255901233886549, 0.004213342221547684, 0.004171208799332207, 0.004129496711338885, 0.004088201744225496, 0.004047319726783242, 0.004006846529515409, 0.003966778064220255, 0.003927110283578052, 0.0038878391807422715, 0.0038489607889348487, 0.0038104711810455, 0.003772366469235045, 0.003734642804542695, 0.0036972963764972675, 0.0036603234127322954, 0.003623720178604972, 0.0035874829768189225, 0.003551608147050733, 0.0035160920655802255, 0.003480931144924423, 0.003446121833475179, 0.0034116606151404273, 0.003377544008989023, 0.0033437685688991327, 0.0033103308832101415, 0.00327722757437804, 0.0032444552986342595, 0.003212010745647917, 0.003179890638191438, 0.003148091731809523, 0.003116610814491428, 0.0030854447063465135, 0.003054590259283049, 0.003024044356690218, 0.0029938039131233154, 0.0029638658739920823, 0.0029342272152521618, 0.0029048849430996404, 0.0028758360936686437, 0.002847077732731957, 0.0028186069554046373, 0.002790420885850591, 0.0027625166769920852, 0.0027348915102221646, 0.0027075425951199428, 0.0026804671691687434, 0.002653662497477056, 0.002627125872502285, 0.0026008546137772626, 0.00257484606763949, 0.002549097606963095, 0.002523606630893464, 0.002498370564584529, 0.002473386858938684, 0.002448652990349297, 0.002424166460445804, 0.002399924795841346, 0.0023759255478829323, 0.0023521662924041033, 0.002328644629480062, 0.002305358183185261, 0.0022823046013534087, 0.002259481555339875, 0.002236886739786476, 0.002214517872388611, 0.002192372693664725, 0.0021704489667280775, 0.002148744477060797, 0.0021272570322901888, 0.002105984461967287, 0.002084924617347614, 0.0020640753711741377, 0.0020434346174623965, 0.0020230002712877727, 0.0020027702685748946, 0.001982742565889146, 0.0019629151402302545, 0.001943285988827952, 0.0019238531289396724, 0.0019046145976502756, 0.0018855684516737728, 0.0018667127671570351, 0.0018480456394854646, 0.0018295651830906102, 0.001811269531259704, 0.0017931568359471069, 0.001775225267587636, 0.0017574730149117594, 0.0017398982847626418, 0.0017224993019150155, 0.0017052743088958652, 0.0016882215658069065, 0.0016713393501488376, 0.001654625956647349, 0.0016380796970808756, 0.0016216989001100668, 0.001605481911108966, 0.0015894270919978766, 0.0015735328210778976, 0.0015577974928671187, 0.0015422195179384474, 0.001526797322759063, 0.0015115293495314723, 0.0014964140560361576, 0.001481449915475796, 0.0014666354163210382, 0.0014519690621578275, 0.0014374493715362492, 0.001423074877820887, 0.001408844129042678, 0.0013947556877522511, 0.0013808081308747287, 0.0013670000495659813, 0.0013533300490703216, 0.0013397967485796185, 0.0013263987810938221, 0.001313134793282884, 0.001300003445350055, 0.0012870034108965545, 0.001274133376787589, 0.0012613920430197129, 0.0012487781225895159, 0.0012362903413636207, 0.0012239274379499845, 0.0012116881635704845, 0.0011995712819347797, 0.001187575569115432, 0.0011756998134242777, 0.001163942815290035, 0.0011523033871371343, 0.001140780353265763, 0.0011293725497331054, 0.0011180788242357745, 0.0011068980359934166, 0.0010958290556334826, 0.0010848707650771476, 0.0010740220574263762, 0.0010632818368521125, 0.0010526490184835913, 0.0010421225282987554, 0.0010317013030157677, 0.0010213842899856101, 0.001011170447085754, 0.0010010587426148964, 0.0009910481551887475, 0.0009811376736368598, 0.0009713262969004913, 0.0009616130339314865, 0.0009519969035921716, 0.0009424769345562499, 0.0009330521652106874, 0.0009237216435585804, 0.0009144844271229945, 0.0009053395828517647, 0.000896286187023247, 0.0008873233251530146, 0.0008784500919014844, 0.0008696655909824696, 0.0008609689350726448, 0.0008523592457219183, 0.0008438356532646992, 0.0008353972967320522, 0.0008270433237647317, 0.0008187728905270843, 0.0008105851616218135, 0.0008024793100055953, 0.0007944545169055393, 0.000786509971736484, 0.0007786448720191191, 0.0007708584232989279, 0.0007631498390659387, 0.0007555183406752793, 0.0007479631572685265, 0.0007404835256958412, 0.0007330786904388828, 0.0007257479035344939, 0.000718490424499149, 0.0007113055202541574, 0.0007041924650516159, 0.0006971505404010997, 0.0006901790349970886, 0.0006832772446471178, 0.0006764444722006466, 0.0006696800274786402, 0.0006629832272038537, 0.0006563533949318153, 0.000649789860982497, 0.0006432919623726722, 0.0006368590427489454, 0.0006304904523214559, 0.0006241855477982414, 0.0006179436923202589, 0.0006117642553970564, 0.0006056466128430858, 0.0005995901467146549, 0.0005935942452475084, 0.0005876583027950332, 0.0005817817197670829, 0.0005759639025694121, 0.0005702042635437179, 0.0005645022209082807, 0.000558857198699198, 0.000553268626712206, 0.0005477359404450839, 0.000542258581040633, 0.0005368359952302268, 0.0005314676352779244, 0.0005261529589251452, 0.0005208914293358938, 0.0005156825150425348, 0.0005105256898921094, 0.0005054204329931884, 0.0005003662286632564, 0.0004953625663766239, 0.0004904089407128577, 0.00048550485130572914, 0.0004806498027926718, 0.00047584330476474506, 0.0004710848717170976, 0.00046637402299992665, 0.00046171028276992737, 0.00045709317994222805, 0.0004525222481428058, 0.00044799702566137775, 0.00044351705540476394, 0.0004390818848507163, 0.00043469106600220914, 0.00043034415534218707, 0.00042604071378876516, 0.0004217803066508775, 0.00041756250358436877, 0.000413386878548525, 0.0004092530097630398, 0.0004051604796654094, 0.0004011088748687553, 0.00039709778612006776, 0.0003931268082588671, 0.0003891955401762784, 0.0003853035847745156, 0.0003814505489267705, 0.00037763604343750277, 0.00037385968300312773, 0.0003701210861730964, 0.0003664198753113654, 0.0003627556765582518, 0.00035912811979266923, 0.0003555368385947426, 0.00035198147020879515, 0.0003484616555067072, 0.00034497703895164015, 0.0003415272685621237, 0.0003381119958765025, 0.0003347308759177375, 0.0003313835671585601, 0.00032806973148697446, 0.00032478903417210473, 0.0003215411438303837, 0.0003183257323920798, 0.00031514247506815905, 0.00031199105031747747, 0.00030887113981430266, 0.00030578242841615964, 0.00030272460413199805, 0.00029969735809067807, 0.0002967003845097713, 0.00029373338066467357, 0.0002907960468580268, 0.0002878880863894465, 0.00028500920552555206, 0.00028215911347029656, 0.0002793375223355936, 0.0002765441471122377, 0.0002737787056411153, 0.00027104091858470413, 0.0002683305093988571, 0.00026564720430486853, 0.0002629907322618198, 0.0002603608249392016, 0.0002577572166898096, 0.00025517964452291146, 0.00025262784807768236, 0.0002501015695969056, 0.0002476005539009365, 0.00024512454836192715, 0.00024267330287830786, 0.00024024656984952475, 0.0002378441041510295, 0.00023546566310951922, 0.00023311100647842404, 0.0002307798964136398, 0.00022847209744950337, 0.00022618737647500832, 0.00022392550271025828, 0.0002216862476831557, 0.00021946938520632414, 0.0002172746913542609, 0.00021510194444071827, 0.00021295092499631107, 0.00021082141574634797, 0.00020871320158888447, 0.00020662606957299562, 0.0002045598088772657, 0.00020251421078849302, 0.0002004890686806081, 0.000198484177993802, 0.00019649933621386398, 0.00019453434285172534, 0.00019258899942320809, 0.00019066310942897599, 0.00018875647833468625, 0.0001868689135513394, 0.00018500022441582598, 0.00018315022217166773, 0.00018131871994995103, 0.00017950553275045153, 0.00017771047742294702, 0.00017593337264871755, 0.00017417403892223038, 0.00017243229853300805, 0.000170707975547678, 0.0001690008957922012, 0.00016731088683427919, 0.0001656377779659364, 0.00016398140018627702, 0.00016234158618441425, 0.0001607181703225701, 0.0001591109886193444, 0.00015751987873315095, 0.00015594467994581947, 0.00015438523314636126, 0.00015284138081489762, 0.00015131296700674867, 0.0001497998373366812, 0.00014830183896331436, 0.00014681882057368122, 0.0001453506323679444, 0.00014389712604426496, 0.0001424581547838223, 0.0001410335732359841, 0.00013962323750362423, 0.000138227005128588, 0.00013684473507730212, 0.0001354762877265291, 0.0001341215248492638, 0.00013278030960077117, 0.00013145250650476345, 0.00013013798143971582, 0.00012883660162531867, 0.00012754823560906547, 0.0001262727532529748, 0.00012501002572044506, 0.0001237599254632406, 0.0001225223262086082, 0.00012129710294652213, 0.0001200841319170569, 0.00011888329059788632, 0.00011769445769190746, 0.00011651751311498839, 0.00011535233798383851, 0.0001141988146040001, 0.00011305682645796012, 0.0001119262581933805, 0.00011080699561144672, 0.00010969892565533224, 0.00010860193639877892, 0.00010751591703479112, 0.00010644075786444322, 0.00010537635028579878, 0.0001043225867829408, 0.0001032793609151114, 0.00010224656730596028, 0.00010122410163290065, 0.00010021186061657165]

## calculation zone

In [131]:
#### Simulate different PIC designs from FL1

# create distance array
# R=0.99, plane_n=459
distance0 = 1500 * 1.46  # um #used glass 0.52um refractive index
distance0 = 5  # um #used glass 0.52um refractive index

# distance0=5 #um
use_array = 1
plane_n = 1
plane_distance = 0  # um
distance_array = [distance0 + plane_distance * i for i in range(plane_n)]
# print(f"distance_array={distance_array}")
print("len(distance_array)=", len(distance_array))
len(distance_array)

# sum_distance2=sum_electric_field_calculator(distance_array,wv_sim_group,wv_sim_num_group,linewidth_group,intensity_arr66,use_array)
# plot_contrast_and_save(sum_distance2,plane_n,plane_distance,distance0,use_array,1,randomness,pitch_x,str(size_img)+"um_image_size")

# #calculation results for different random sample
use_array = 0  # whether use the intensity array for different z planes in the cavity
# for randomness in [0.01 / 2, 0.05 / 2, 0.1 / 2, 0.25 / 2]:
print(pitch_x)
for randomness in [0.05 / 2]:
    randx = (
        randomness * pitch_x
    )  # um? #the amplitude of the random displacement along x.
    rand_x_list = (
        np.random.rand(rep_num_y, rep_num_x) * randx * 2 - randx
    )  # Random x-displacement for each element; hyperuniform
    rand_y_list = (np.random.rand(rep_num_y, 1) * randx * 2 - randx).repeat(
        rep_num_x, axis=1
    )  # Random y-displacement per row
    for i in range(rep_num_y):
        for j in range(rep_num_x):
            amp[Ly * i : Ly * (i + 1), Lx * j : Lx * (j + 1)] = generate_2d_gaussian(
                emitter_size / delta,
                [rand_y_list[i, j] / delta, rand_x_list[i, j] / delta],
                [Ly, Lx],
            )
    amp_pad = np.pad(amp, pad_width=padding)

    figsize = (5, 5)
    plt.figure(figsize=figsize)
    plt.imshow(amp, cmap="gray")
    plt.show()
    figsize = (5, 5)
    plt.figure(figsize=figsize)
    plt.imshow(amp_pad, cmap="gray")
    plt.show()

    sum_distance2 = sum_electric_field_calculator(
        distance_array,
        wv_sim_group,
        wv_sim_num_group,
        linewidth_group,
        intensity_arr66,
        use_array,
    )
    plot_contrast_and_save(
        sum_distance2,
        plane_n,
        plane_distance,
        distance0,
        use_array,
        1,
        randomness,
        pitch_x,
        str(size_img) + "um_image_size",
    )


In [23]:
#create distance array
#R=0.99, plane_n=459
distance0=500 #um
plane_n=459
plane_distance=150 #um
distance_array=[distance0+plane_distance*i for i in range(plane_n)]
#print(f"distance_array={distance_array}")
print("len(distance_array)=",len(distance_array))
len(distance_array)

#calculation results
use_array=1
sum_distance2=sum_electric_field_calculator(distance_array,wv_sim_group,wv_sim_num_group,linewidth_group,intensity_arr459R99,use_array)
plot_contrast_and_save(sum_distance2,plane_n,plane_distance,distance0,use_array,1)

In [332]:
#create distance array
distance0=8000 #um
plane_n=2
plane_distance=150 #um
distance_array=[distance0+plane_distance*i for i in range(plane_n)]
#print(f"distance_array={distance_array}")
print("len(distance_array)=",len(distance_array))
len(distance_array)

#calculation results
use_array=1
sum_distance2=sum_intensity_calculator(distance_array,wv_sim_group,wv_sim_num_group,linewidth_group,intensity_arr66,use_array)
plot_contrast_and_save(sum_distance2,plane_n,plane_distance,distance0,use_array,0)

In [320]:
# create distance array
distance0 = 8000  # um
plane_n = 1
plane_distance = 150  # um
distance_array = [distance0 + plane_distance * i for i in range(plane_n)]
# print(f"distance_array={distance_array}")
print("len(distance_array)=", len(distance_array))
len(distance_array)

# calculation results
use_array = 1
sum_distance2 = sum_intensity_calculator(
    distance_array,
    wv_sim_group,
    wv_sim_num_group,
    linewidth_group,
    intensity_arr66,
    use_array,
)
plot_contrast_and_save(sum_distance2, plane_n, plane_distance, distance0, use_array,0)

In [211]:
#create distance array
distance0=8000 #um
plane_n=66
plane_distance=150 #um
distance_array=[distance0+plane_distance*i for i in range(plane_n)]
#print(distance_array)
print(len(distance_array))

#calculation results
#sum_distanceold=sum_intensity_calculator(distance_array,wv_sim_group,wv_sim_num_group,linewidth_group)
plot_contrast_and_save(sum_distance,plane_n,plane_distance,distance0)

## plot comparison zone

In [3]:
#normalize to the same sum power

from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

def scale_image_to_target_sum(image_path, target_per_pixel):
    # Load and convert image to grayscale
    image = Image.open(image_path).convert('L')
    img_array = np.array(image)
    # Calculate scaling factor
    num_pixels = img_array.size
    desired_sum = num_pixels * target_per_pixel
    current_sum = img_array.sum()
    scale_factor = desired_sum / current_sum
    # Scale the image
    scaled_img_array = img_array * scale_factor
    # Check if the maximum value is greater than 255
    if scaled_img_array.max() > 255:
        # # Adjust target_per_pixel to avoid exceeding the maximum pixel value of 255
        # target_per_pixel = 255 * current_sum / num_pixels
        # scale_factor = target_per_pixel / current_sum
        # scaled_img_array = img_array * scale_factor
        print("overfill,need to decrease target_per_pixel")
        print("!!!!!!!")
        print("!!!!!!!")
        print("!!!!!!!")
        print("!!!!!!!")
    power=scaled_img_array.sum()/img_array.size
    # Convert scaled image array to uint8
    scaled_img_array = scaled_img_array.astype(np.uint8)
    # Convert back to PIL image
    scaled_image = Image.fromarray(scaled_img_array)
    return image, scaled_image, power  # Return both original and scaled images

# Define paths and target brightness per pixel
image1_path = 'sum_electric_field=1_459plane_150umdistance_8000umdistance0_1intensityarray_contrast0.12889.png'
image2_path = '1plane_150umdistance_8000umdistance0_1intensityarray_contrast0.34373.png'
target_brightness_per_pixel = 70

# Process images
original_image1, scaled_image1,power1 = scale_image_to_target_sum(image1_path, target_brightness_per_pixel)
original_image2, scaled_image2,power2 = scale_image_to_target_sum(image2_path, target_brightness_per_pixel)

# Function to display images
def display_images(original, scaled, title):
    plt.figure(figsize=(10, 5))
    plt.subplot(1, 2, 1)
    plt.imshow(original, cmap='gray',vmin=0, vmax=255)
    plt.title('Original ' + title)
    plt.axis('off')
    
    plt.subplot(1, 2, 2)
    plt.imshow(scaled, cmap='gray',vmin=0, vmax=255)
    plt.title('Scaled ' + title)
    plt.axis('off')
    
    plt.show()

# Display both sets of images
display_images(original_image1, scaled_image1, "image1")
display_images(original_image2, scaled_image2, "image2")
print(" power1=",power1, ",power2= ",power2)

In [ ]:
# upload to manifold

# Parameters (dims in um)
wavelengths = {
    "r": [0.638],
    "g": [0.520],
    "b": [0.450],
}

sampling_res = delta
sampling_img_size = len(us_group['g'][0])
sampling_npix = 2 * np.floor(sampling_img_size / sampling_res / 2)

x = sampling_npix * np.linspace(
    -sampling_res / 2, sampling_res / 2, sampling_npix.astype(int)
)
y = sampling_npix * np.linspace(
    -sampling_res / 2, sampling_res / 2, sampling_npix.astype(int)
)
xg, yg = np.meshgrid(x, y)

## Form h5 file with the relevant data
id = "X20um_Y80um_Gaussian1nm_period{}".format(peroid)  # ID for the simulation
file_name = "speckle_field_data_{}.h5".format(id)
path = "tree/epic/speckle/dynamic/activeLC/1nmGaussian/{}".format(file_name)

with h5py.File(file_name, "w") as h5:
    h5["sampling_res"] = sampling_res
    h5["sampling_npix"] = sampling_npix
    h5["sampling_img_size"] = sampling_img_size
    h5["x"] = x
    h5["y"] = y

    fields = h5.create_group("fields")
    for c, color in enumerate(wavelengths):
        color_set = h5.create_group(str(color))
        for w, wavelength in enumerate(wavelengths[color]):
            wavelength_set = h5.create_group(str(wavelength))
            us_list = us_group[color]
            for idx in range(len(us_list)):
                field = us_list[idx, :, :]
                wavelength_set[str(idx)] = field
            color_set[str(wavelength)] = wavelength_set
        fields[str(color)] = color_set
## Save file to Manifold (folder structure created automatically if not existing)
storage_async.manifold_upload_file(path=path, localfile=file_name, overwrite=True)